In [ ]:
# The below codes are heavily inspired by Surprise official documentation
# https://github.com/NicolasHug/Surprise/tree/9a263a85decc4236919b4f8a706783d7e8fcad24/surprise

In [ ]:
#Dependencies : 
# Original data : User+Movie+rating
# Environmemnt with working python libraries

#Outputs of the codes
# Dictionary Files for userIDs and MovieIDs
# Processed Data file
# Train/Test Data Files
# Model Pickle

In [ ]:
import time

In [ ]:
%pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633997 sha256=722c0cfdd9ec51a4ef2a487895d19e973ff0dcc2a64d37e7c6809320b9852b2f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DataPreProcessing.py

import pandas as pd

def DataPreProcessing(file_path="/content/drive/MyDrive/Collab Codes/MLIP/kafka_ratings.txt",save_as="/content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv"):
  df=pd.read_csv("/content/drive/MyDrive/Collab Codes/MLIP/kafka_ratings.txt",header=None)
  
  df.columns=["Timestamp","UserID","Log"]

  df["MovieName=Rating"]=df["Log"].apply(lambda x: x.split("/")[2])
  df["MovieName"]=df["MovieName=Rating"].apply(lambda x: x.split("=")[0])
  df["rating"]=df["MovieName=Rating"].apply(lambda x: x.split("=")[1])
  
  Data=df[["UserID","MovieName","rating","Timestamp"]]
  
  
  # Data sort on timestamp

  # Few duplicates of user-> movie were dropped, the lateset was picked
  Data=Data.drop_duplicates(subset=['UserID', 'MovieName'],keep="last")
  print("Data.shape",Data.shape)
  
  #Exception handling for data save
  try : 
    Data.to_csv(save_as)
    print("Saved at: ",save_as)
    return True
  except:
    print("Unable to save at: ",save_as)
    return False

In [ ]:
from collections import defaultdict
import json

def make_Dictionaries(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv",dictionary_storage_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  Data=pd.read_csv(processed_data_file)

    
  # Function to return a default
  # values for keys that is not
  # present
  def def_value():
      return "Not Present"
        
  # Defining the dict
  iid_dict = defaultdict(def_value)
  uid_dict = defaultdict(def_value)
  sort_movies=Data['MovieName'].unique()
  sort_movies.sort()
  for i in range(len(sort_movies)):
    iid_dict[sort_movies[i]]=i

  sort_users=Data['UserID'].unique()
  sort_users.sort()
  for i in range(len(sort_users)):
    uid_dict[int(sort_users[i])]=i

  with open(dictionary_storage_location+"iid_dict.json", "w") as outfile:
    json.dump(iid_dict, outfile)
  with open(dictionary_storage_location+"uid_dict.json", "w") as outfile:
    json.dump(uid_dict, outfile)

  return True


In [ ]:
# importing the module
import json
 
def DataRemapping(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv",dictionary_storage_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  # Opening JSON file
  with open(dictionary_storage_location+'iid_dict.json') as json_file:
      iid_dict = json.load(json_file)
  with open(dictionary_storage_location+'uid_dict.json') as json_file:
      uid_dict = json.load(json_file)
  def def_value():
      return "Not Present"
        
  # Defining the dict
  iid_dict = defaultdict(int,iid_dict)
  uid_dict = defaultdict(int,uid_dict)
  
  Data=pd.read_csv(processed_data_file)
  Data['item']=Data['MovieName'].apply(lambda x: iid_dict[x])
  Data['user']=Data['UserID'].apply(lambda x: uid_dict[str(x)])
  # print(Data.head())
  Data_selected=Data[['user','item','rating']]
  # print(Data_selected.head())
  Data_selected.to_csv(dictionary_storage_location+"RemappedData.csv")
  
  User_watched_movies=Data_selected.groupby('user')['item'].apply(list)
  User_watched_movies=dict(User_watched_movies)
  with open(dictionary_storage_location+"Users_Watched_Movies.json", "w") as outfile:
    json.dump(User_watched_movies, outfile)
  
  return True

In [ ]:
from surprise import Reader
from surprise import NMF
from surprise import Dataset
import pandas as pd
from surprise.accuracy import rmse
from surprise.model_selection import KFold
import numpy as np
def train_nmf(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/RemappedData.csv"):
  
  Data_selected=pd.read_csv(processed_data_file)

  reader = Reader(rating_scale=(1, 5))

  # Loads Pandas dataframe
  data = Dataset.load_from_df(Data_selected[["user", "item", "rating"]], reader)



  """
  lr_all is the learning rate for all parameters (how much the parameters are adjusted in each iteration)
  reg_all is the regularization term for all parameters, which is a penalty term added to prevent overfitting.
  """

  param_grid = {
      "n_epochs": [5, 10],
      "lr_all": [0.002, 0.005],
      "reg_all": [0.4, 0.6]
  }

  # Get the best params using GridSearchCV
  # gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)
  # gs.fit(data)
  # Post GS we have the below params
  # best_params=best_params = gs.best_params["rmse"]
  best_params={'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
  # print("BEST PARAM:",best_params)
  # Extract and train model with best params
  svd_algo = NMF()
  # Train
  trainingSet = data.build_full_trainset()


  svd_algo.fit(trainingSet)

  kf = KFold(n_splits=3)
  errors=[]
  for trainset, testset in kf.split(data):
    svd_algo.fit(trainset)                             
    predictions = svd_algo.test(testset)
    errors.append(rmse(predictions))

  return svd_algo,np.mean(errors)

In [ ]:
from surprise import Reader
from surprise import SVDpp
from surprise import Dataset
import pandas as pd
from surprise.accuracy import rmse
from surprise.model_selection import KFold
import numpy as np
def train_svdpp(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/RemappedData.csv"):
  
  Data_selected=pd.read_csv(processed_data_file)

  reader = Reader(rating_scale=(1, 5))

  # Loads Pandas dataframe
  data = Dataset.load_from_df(Data_selected[["user", "item", "rating"]], reader)



  """
  lr_all is the learning rate for all parameters (how much the parameters are adjusted in each iteration)
  reg_all is the regularization term for all parameters, which is a penalty term added to prevent overfitting.
  """

  param_grid = {
      "n_epochs": [5, 10],
      "lr_all": [0.002, 0.005],
      "reg_all": [0.4, 0.6]
  }

  # Get the best params using GridSearchCV
  # gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)
  # gs.fit(data)
  # Post GS we have the below params
  # best_params=best_params = gs.best_params["rmse"]
  best_params={'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
  # print("BEST PARAM:",best_params)
  # Extract and train model with best params
  svd_algo = SVDpp()
  # Train
  trainingSet = data.build_full_trainset()


  svd_algo.fit(trainingSet)

  kf = KFold(n_splits=3)
  errors=[]
  for trainset, testset in kf.split(data):
    svd_algo.fit(trainset)                             
    predictions = svd_algo.test(testset)
    errors.append(rmse(predictions))

  return svd_algo,np.mean(errors)

In [ ]:
from surprise import Reader
from surprise import Dataset
import pandas as pd
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse
from surprise.model_selection import KFold


def train_SVD_wGridSearch(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/RemappedData.csv"):
  
  Data_selected=pd.read_csv(processed_data_file)

  reader = Reader(rating_scale=(1, 5))

  # Loads Pandas dataframe
  data = Dataset.load_from_df(Data_selected[["user", "item", "rating"]], reader)



  """
  lr_all is the learning rate for all parameters (how much the parameters are adjusted in each iteration)
  reg_all is the regularization term for all parameters, which is a penalty term added to prevent overfitting.
  """

  param_grid = {
      "n_epochs": [5, 10],
      "lr_all": [0.002, 0.005],
      "reg_all": [0.4, 0.6]
  }

  # Get the best params using GridSearchCV
  # gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)
  # gs.fit(data)
  # Post GS we have the below params
  # best_params=best_params = gs.best_params["rmse"]
  best_params={'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
  # print("BEST PARAM:",best_params)
  # Extract and train model with best params
  svd_algo = SVD(n_epochs=best_params['n_epochs'],
                lr_all=best_params['lr_all'],
                reg_all=best_params['reg_all'])
  # Train
  trainingSet = data.build_full_trainset()


  svd_algo.fit(trainingSet)

  kf = KFold(n_splits=3)
  errors=[]
  for trainset, testset in kf.split(data):
    svd_algo.fit(trainset)                             
    predictions = svd_algo.test(testset)
    errors.append(rmse(predictions))

  return svd_algo,np.mean(errors)

In [ ]:
# the predict function for any algorithm
def predict(algo,uid,iid,r=4):
  ret=algo.predict(int(uid),int(iid),r)
  # print(ret)
  return ret

In [ ]:
# storing and retreiving model objects

import pickle 

def store_model_into_pickle(algo,algo_name,dest_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  file1 = open(dest_location+algo_name+".pickle", "wb")
  pickle.dump(algo, file1)
  file1.close()
  print(algo_name+" Stored at/as :"+dest_location+algo_name+".pickle")
  return True
  
def get_model_from_pickle(algo_name,file_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  file_to_read = open(file_location+algo_name+".pickle", "rb")
  loaded_object = pickle.load(file_to_read)
  file_to_read.close()
  print("returned model object")
  return loaded_object



In [ ]:
from operator import itemgetter

def recommend_top_20_modviesForUser(algo_name,og_user,dictionary_storage_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  og_user=str(og_user)
  algo=get_model_from_pickle(algo_name=algo_name)
  with open(dictionary_storage_location+'iid_dict.json') as json_file:
      iid_dict = json.load(json_file)
  with open(dictionary_storage_location+'uid_dict.json') as json_file:
      uid_dict = json.load(json_file)
  with open(dictionary_storage_location+'Users_Watched_Movies.json') as json_file:
      Users_Watched_Movies = json.load(json_file)
  def def_value():
      return "Not Present"
        
  # Defining the dict
  iid_dict = defaultdict(int,iid_dict)
  uid_dict = defaultdict(int,uid_dict)

  #Defining inverted dictionaries
  inv_uid = {v: k for k, v in uid_dict.items()}
  inv_iid = {v: k for k, v in iid_dict.items()}


  model_uid=str(uid_dict[og_user])
  print("MODEL UID:",model_uid,"OG ID",og_user)
  all_movies_user_saw=Users_Watched_Movies[model_uid]

  setA = set(list(iid_dict.values()))
  setB = set(all_movies_user_saw)

  # Removing movies which user has already seen
  onlyInA = setA.difference(setB)
  predictions=[]
  for movie in onlyInA:
    predictions.append(predict(algo,model_uid,movie)[3])

  preds_for_a_user=dict(zip(onlyInA,predictions))
  top20movieIDs = dict(sorted(preds_for_a_user.items(), key = itemgetter(1), reverse = True)[:20])
  # print(top20movieIDs)
  top20movieNames=[inv_iid[i] for i in list(top20movieIDs.keys())]
  out_json={"message": "OK", "recommendations": top20movieNames }
  return out_json

In [ ]:
# Code to get a model's performace, wrt : Avg. Training time and RMSE on a K-Folded Cross Validation Process
# Assumption that a model has been trained earlier, however, this below process only uses the architecture and not the trained model. It will re-fit the data again.
# Hence, Data Location is essential.
# The purpose of this code is only model evaualtion for the sake of seperation of concerns.

from surprise import Reader
from surprise import NMF
from surprise import SVDpp
from surprise import SVD
from surprise import Dataset
from surprise.accuracy import rmse
from surprise.model_selection import KFold
import numpy as np
import pandas as pd
import pickle 
import requests

#Please update your desitination location if its different

#Codes to store the model into pickle 
def store_model_into_pickle(algo,algo_name,dest_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  file1 = open(dest_location+algo_name+".pickle", "wb")
  pickle.dump(algo, file1)
  file1.close()
  print(algo_name+" Stored at/as :"+dest_location+algo_name+".pickle")
  return True

#Codes to get the model from pickle 
def get_model_from_pickle(algo_name,file_location="/content/drive/MyDrive/Collab Codes/MLIP/"):
  file_to_read = open(file_location+algo_name+".pickle", "rb")
  loaded_object = pickle.load(file_to_read)
  file_to_read.close()
  print("returned model object")
  return loaded_object



def get_avg_throughputtime(data_location="/content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv"):
  df=pd.read_csv(data_location)
  request_list=['http://128.2.205.106:8082/recommend/'+str(i) for i in np.unique(df.UserID.sample(500))]

  times=[]
  # Making a get request
  for i in request_list:
    response = requests.get(i)
    times.append(response.elapsed.total_seconds())

  print("AVG Throughput",np.mean(times))
  return np.mean(times)

# An existing model architechture chan be passed to the below code, and the code will re-fit the model on the k-folded data and give the most realistic k-fold cross validation RMSE
def evaluate_model(model_name="svdpp_model",processed__test_data_file="/content/drive/MyDrive/Collab Codes/MLIP/RemappedData.csv"):

  algo=get_model_from_pickle(model_name)
  Data_selected=pd.read_csv(processed__test_data_file)
  reader = Reader(rating_scale=(1, 5))

  # Loads Pandas dataframe
  data = Dataset.load_from_df(Data_selected[["user", "item", "rating"]], reader)

  #KFold helps us to spit the dataset into n folds, this will help us seperate training and testing splits while calculating testing erros

  kf = KFold(n_splits=4)
  errors=[]
  training_time=[]
  for trainset, testset in kf.split(data):
    start_time = time.time()
    #Fitting
    algo.fit(trainset)      
    end_time=time.time()               
    time_for_fitting=(time.time() - start_time)

    # we are checking for test rmse only, and computing avg of all CV in the k splits
    predictions = algo.test(testset)
    #appending results to a list for future references
    errors.append(rmse(predictions))
    training_time.append(time_for_fitting)
    
  # This code only returns the avg RMSE error and avg training time for a k folded dataset
  return np.mean(errors),np.mean(training_time)

In [ ]:
# FINAL SEQUENTIAL EXECUTION ONE TIME
start_time = time.time()
DataPreProcessing()
end_time=time.time()
time_for_dataProcessing=(time.time() - start_time)

df=pd.read_csv("/content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv")
sampled_df=df.sample(170000)
sampled_df.to_csv("/content/drive/MyDrive/Collab Codes/MLIP/sampled_170kpreprocessed.csv")


Data.shape (435452, 4)
Saved at:  /content/drive/MyDrive/Collab Codes/MLIP/preprocessed.csv


In [ ]:
start_time = time.time()
make_Dictionaries(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/sampled_20kpreprocessed.csv")
DataRemapping(processed_data_file="/content/drive/MyDrive/Collab Codes/MLIP/sampled_20kpreprocessed.csv")
end_time=time.time()
time_for_dataReMapping=(time.time() - start_time)

start_time = time.time()
svdpp_algo,avgerror_svdpp=train_svdpp()
end_time=time.time()
time_for_svdpp=(time.time() - start_time)

start_time = time.time()
nmf_algo,avgerror_nmf=train_nmf()
end_time=time.time()
time_for_nmf=(time.time() - start_time)

start_time = time.time()
svd_algo,avgerror_svd=train_SVD_wGridSearch()
end_time=time.time()
time_for_SVD=(time.time() - start_time)

RMSE: 0.9557
RMSE: 0.9478
RMSE: 0.9551
RMSE: 1.1407
RMSE: 1.1437
RMSE: 1.1408
RMSE: 0.9604
RMSE: 0.9575
RMSE: 0.9591


In [ ]:
# FINAL SEQUENTIAL EXECUTION ONE TIME
store_model_into_pickle(nmf_algo,"nmf_model")
nmf_algo=get_model_from_pickle(algo_name="nmf_model")
store_model_into_pickle(svdpp_algo,"svdpp_model")
svdpp_algo=get_model_from_pickle(algo_name="svdpp_model")
store_model_into_pickle(svd_algo,"svd_model")
svd_algo=get_model_from_pickle(algo_name="svd_model")

nmf_model Stored at/as :/content/drive/MyDrive/Collab Codes/MLIP/nmf_model.pickle
returned model object
svdpp_model Stored at/as :/content/drive/MyDrive/Collab Codes/MLIP/svdpp_model.pickle
returned model object
svd_model Stored at/as :/content/drive/MyDrive/Collab Codes/MLIP/svd_model.pickle
returned model object


In [ ]:

start_time = time.time()
avgerror_svdpp,training_time_svdpp=evaluate_model("svdpp_model")
end_time=time.time()
time_for_svdpp=(time.time() - start_time)

start_time = time.time()
avgerror_nmf,training_time_nmf=evaluate_model("nmf_model")
end_time=time.time()
time_for_nmf=(time.time() - start_time)

start_time = time.time()
avgerror_svd,training_time_svd=evaluate_model("svd_model")
end_time=time.time()
time_for_SVD=(time.time() - start_time)

returned model object
RMSE: 0.9546
RMSE: 0.9473
RMSE: 0.9453
RMSE: 0.9480
returned model object
RMSE: 1.1284
RMSE: 1.1378
RMSE: 1.1281
RMSE: 1.1303
returned model object
RMSE: 0.9525
RMSE: 0.9584
RMSE: 0.9570
RMSE: 0.9577


In [ ]:
#API RETURN , MANY TIMES ACCESS
start_time = time.time()
temp=recommend_top_20_modviesForUser("svd_model",63233)
end_time=time.time()
time_for_SVD_pred=(time.time() - start_time)

start_time = time.time()
temp=recommend_top_20_modviesForUser("svdpp_model",63233)
end_time=time.time()
time_for_SVDPP_pred=(time.time() - start_time)

start_time = time.time()
temp=recommend_top_20_modviesForUser("nmf_model",63233)
end_time=time.time()
time_for_NMF_pred=(time.time() - start_time)

returned model object
MODEL UID: 14202 OG ID 63233
returned model object
MODEL UID: 14202 OG ID 63233
returned model object
MODEL UID: 14202 OG ID 63233


In [ ]:
import sys,os

print("MODEL AVG RMSE ERROR after 3 fold CV SVD:",avgerror_svd)
print("MODEL AVG RMSE ERROR after 3 fold CV SVDPP:",avgerror_svdpp)
print("MODEL AVG RMSE ERROR after 3 fold CV NMF:",avgerror_nmf)
print()

print("(seconds) Time for Data Processing: ",time_for_dataProcessing)
print("(seconds) Time for Data Remapping: ",time_for_dataReMapping)
print()
print("(seconds) Time for Model Training (SVD): ",training_time_svd)
print("(seconds) Time for Model Training (SVDPP): ",training_time_svdpp)
print("(seconds) Time for Model Training (NMF): ",training_time_nmf)
print()
print("(seconds) Time for Model Prediction (SVD): ",time_for_SVD_pred)
print("(seconds) Time for Model Prediction (SVDPP): ",time_for_SVDPP_pred)
print("(seconds) Time for Model Prediction (NMF): ",time_for_NMF_pred)
print()
print("Storage Size for Model Pickle (SVD) in MB: ",(os.stat("/content/drive/MyDrive/Collab Codes/MLIP/svd_model.pickle").st_size / (1024 * 1024)))
print("Storage Size for Model Pickle (SVDPP) in MB: ",(os.stat("/content/drive/MyDrive/Collab Codes/MLIP/svdpp_model.pickle").st_size / (1024 * 1024)))
print("Storage Size for Model Pickle (NMF) in MB: ",(os.stat("/content/drive/MyDrive/Collab Codes/MLIP/nmf_model.pickle").st_size / (1024 * 1024)))




MODEL AVG RMSE ERROR after 3 fold CV SVD: 0.9564021739184455
MODEL AVG RMSE ERROR after 3 fold CV SVDPP: 0.9487898160307124
MODEL AVG RMSE ERROR after 3 fold CV NMF: 1.1311381396938485

(seconds) Time for Data Processing:  2.2956197261810303
(seconds) Time for Data Remapping:  2.2296266555786133

(seconds) Time for Model Training (SVD):  3.9703701734542847
(seconds) Time for Model Training (SVDPP):  13.818799376487732
(seconds) Time for Model Training (NMF):  12.65781307220459

(seconds) Time for Model Prediction (SVD):  0.723773717880249
(seconds) Time for Model Prediction (SVDPP):  0.6875138282775879
(seconds) Time for Model Prediction (NMF):  0.5831031799316406

Storage Size for Model Pickle (SVD) in MB:  47.571370124816895
Storage Size for Model Pickle (SVDPP) in MB:  15.210856437683105
Storage Size for Model Pickle (NMF) in MB:  11.146574020385742


In [ ]:
sampled_df=pd.read_csv("/content/drive/MyDrive/Collab Codes/MLIP/sampled_20kpreprocessed.csv")
sampled_df

,Unnamed: 0,Unnamed: 0.1,UserID,MovieName,rating,Timestamp
0,367672,367733,158998,raiders+of+the+lost+ark+1981,5,2022-09-08T13:11:40
1,376673,376734,195350,robin+hood+prince+of+thieves+1991,4,2022-09-08T16:05:52
2,44801,44808,92730,forrest+gump+1994,5,2022-08-28T18:51:11
3,34986,34991,137202,superbad+2007,4,2022-08-14T05:25:37
4,273867,273910,67535,vanilla+sky+2001,3,2022-08-28T05:42:15
...,...,...,...,...,...,...
149995,327488,327538,219017,back+to+the+future+1985,3,2022-08-14T20:03:54
149996,393082,393146,144962,three+days+2008,2,2022-09-08T19:22:59
149997,101453,101467,120517,amores+perros+2000,4,2022-09-02T00:41:08
149998,175149,175176,215113,happy+gilmore+1996,4,2022-08-30T20:36:06


In [ ]:
#API RETURN , MANY TIMES ACCESS
recommend_top_20_modviesForUser("svd_model",63233)

returned model object
MODEL UID: 14202 OG ID 63233


{'message': 'OK',
 'recommendations': ['the+shawshank+redemption+1994',
  'the+dark+knight+rises+2012',
  'the+lives+of+others+2006',
  'the+godfather+1972',
  'the+philadelphia+story+1940',
  'rebecca+1940',
  'the+usual+suspects+1995',
  'm+1931',
  'annie+hall+1977',
  'double+indemnity+1944',
  'pride++prejudice+2005',
  'the+wrong+trousers+1993',
  'secrets++lies+1996',
  'rear+window+1954',
  'snatch+2000',
  'the+godfather+part+ii+1974',
  'monty+python+and+the+holy+grail+1975',
  'schindlers+list+1993',
  'spirited+away+2001',
  'one+flew+over+the+cuckoos+nest+1975']}

In [ ]:
#API RETURN , MANY TIMES ACCESS
recommend_top_20_modviesForUser("svd_model",78444)

returned model object
MODEL UID: 17662 OG ID 78444


{'message': 'OK',
 'recommendations': ['the+shawshank+redemption+1994',
  'the+lives+of+others+2006',
  'the+seventh+seal+1957',
  'the+godfather+1972',
  '8+1963',
  'snatch+2000',
  'the+philadelphia+story+1940',
  'the+usual+suspects+1995',
  'spirited+away+2001',
  'annie+hall+1977',
  'schindlers+list+1993',
  'the+wrong+trousers+1993',
  'the+graduate+1967',
  'crumb+1994',
  'one+flew+over+the+cuckoos+nest+1975',
  'monty+python+and+the+holy+grail+1975',
  '12+angry+men+1957',
  'lone+star+1996',
  'seven+samurai+1954',
  'fight+club+1999']}

In [ ]:
#API RETURN , MANY TIMES ACCESS
recommend_top_20_modviesForUser("knn_model",63233)

returned model object
MODEL UID: 14202 OG ID 63233


{'message': 'OK',
 'recommendations': ['saving+face+2004',
  'one+lucky+elephant+2011',
  'bigger+than+life+1956',
  'dragon+day+2013',
  'what+a+girl+wants+2003',
  'antonio+das+mortes+1969',
  'the+secret+of+roan+inish+1994',
  'thieves+1996',
  'moonlight+murder+1936',
  'cinderella+1950',
  'harvie+krumpet+2003',
  'cross+of+iron+1977',
  'the+long+goodbye+1973',
  'the+spongebob+movie+sponge+out+of+water+2015',
  'shattered+glass+2003',
  'cj7+2008',
  'the+long+kiss+goodnight+1996',
  'wonderland+1999',
  'ratcatcher+1999',
  'the+boondock+saints+ii+all+saints+day+2009']}

In [ ]:
#API RETURN , MANY TIMES ACCESS
recommend_top_20_modviesForUser("knn_model",78444)

returned model object
MODEL UID: 17662 OG ID 78444


{'message': 'OK',
 'recommendations': ['primal+fear+1996',
  'the+tao+of+steve+2000',
  'one+lucky+elephant+2011',
  'catlow+1971',
  'mr.+warmth+the+don+rickles+project+2007',
  'the+secret+life+of+bees+2008',
  'vixen+1968',
  'alice+in+wonderland+2010',
  'selena+1997',
  'ratcatcher+1999',
  'it+might+get+loud+2009',
  'north+1994',
  'inside+deep+throat+2005',
  'great+freedom+no.+7+1944',
  '200+cigarettes+1999',
  'the+champ+1979',
  '12+angry+men+1997',
  'bigger+than+life+1956',
  'thieves+1996',
  'altered+2006']}